In [13]:
# Import libraries
import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px
from IPython.core.display import HTML

In [15]:
# Page configuration
st.set_page_config(
    page_title="Import-Export Dashboard",
    page_icon="📦",
    layout="wide",
    initial_sidebar_state="expanded")

In [17]:
page_bg_color = """
    <style>
    body {
        background-color: #e8f5e9;
    }
    .metric-container {
        display: flex;
        justify-content: space-between;
    }
    .metric-box {
        padding: 15px;
        margin: 5px;
        border-radius: 10px;
        background-color: #ffffff;
        box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1);
        text-align: center;
    }
    .metric-title {
        font-size: 16px;
        font-weight: bold;
        color: #333333;
    }
    .metric-value {
        font-size: 20px;
        color: #007bff;
    }
    .metric-delta {
        color: #ff6b6b;
    }
    </style>
"""

In [19]:
st.markdown(page_bg_color, unsafe_allow_html=True)

alt.themes.enable("dark")

ThemeRegistry.enable('dark')

In [21]:
# Load data
df = pd.read_csv("C:/Users/jagri/OneDrive/Desktop/Imports_Exports_Dataset.csv")
df

,Transaction_ID,Country,Product,Import_Export,Quantity,Value,Date,Category,Port,Customs_Code,Weight,Shipping_Method,Supplier,Customer,Invoice_Number,Payment_Terms
0,e3e70682-c209-4cac-a29f-6fbed82c07cd,Colombia,describe,Export,1979,9506.57,07-12-2023,Machinery,Robertbury,620537,4248.65,Air,"Garrison, Hubbard and Hendricks",Seth Hall,21000294,Cash on Delivery
1,f728b4fa-4248-4e3a-8a5d-2f346baa9455,Chile,president,Export,5763,7100.91,04-04-2023,Clothing,Shahport,927600,4340.81,Air,Webb-Mack,Kimberly Ryan,88738432,Prepaid
2,eb1167b3-67a9-4378-bc65-c1e582e2e662,Turkey,far,Import,5740,2450.34,21-08-2024,Electronics,South Joshuatown,299258,4538.41,Air,"Mendez, Jones and Johnson",Ryan Silva,89922099,Prepaid
3,f7c1bd87-4da5-4709-9471-3d60c8a70639,Christmas Island,agency,Export,2592,7226.42,09-05-2022,Furniture,Adamfort,996084,4886.14,Air,Schroeder-Smith,Jacob Gray,63216265,Net 60
4,e443df78-9558-467f-9ba9-1faf7a024204,Finland,policy,Export,2622,2817.29,03-04-2023,Electronics,Juliebury,555981,4406.74,Air,Zimmerman LLC,Amy Stephens,94600248,Cash on Delivery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,48df15a8-0823-4964-8c16-eddf2756f382,Marshall Islands,not,Export,2860,2055.19,09-07-2024,Furniture,South Karenfort,393463,4120.35,Land,Smith-Lewis,Darlene Davis,29605073,Net 60
14996,31106617-94a6-4646-a001-5e7bd45abc26,Bermuda,air,Export,2443,6407.06,18-06-2024,Furniture,Jeffreyside,484143,1832.71,Air,Jones Group,John Ramos,39044695,Cash on Delivery
14997,ee485839-fbde-4ced-af18-d98f5e863081,Tanzania,show,Export,1702,9918.29,30-04-2020,Toys,North Meganborough,354935,4203.52,Land,Barnes-Romero,Rebecca Phelps,78492040,Net 30
14998,5acd54aa-ec8c-4055-be8b-a447861a471c,Tuvalu,TV,Export,8108,9288.57,29-04-2021,Clothing,Villafurt,234296,1597.72,Land,"Smith, Allison and Bennett",Scott Yates,20799602,Cash on Delivery


In [23]:
# Sidebar
with st.sidebar:
    st.title('📦 Import-Export Dashboard')

In [27]:
# Filter by year
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
year_list = sorted(df['Date'].dt.year.unique(), reverse=True)
selected_year = st.selectbox('Select a year', year_list)

In [33]:
# Filter data based on selected year
df_selected_year = df[df['Date'].dt.year == selected_year]

In [39]:
# Multi-select for country
country_list = df['Country'].unique()
selected_countries = st.multiselect('Select country(s)', country_list, default=country_list[:3])  # Default first 3

# Ensure correct indentation here
if selected_countries:
    df_selected_year = df_selected_year[df_selected_year['Country'].isin(selected_countries)]

In [43]:
 # Select color theme
color_theme_list = ['blues', 'reds', 'greens', 'turbo', 'viridis']
selected_color_theme = st.selectbox('Select a color theme', color_theme_list)

In [45]:
# Helper Functions
def format_value(val):
    return f"${val / 1e6:.1f}M" if val > 1e6 else f"${val / 1e3:.1f}K"

In [49]:
def calculate_transaction_difference(input_df, input_year):
    selected_year_data = input_df[input_df['Date'].dt.year == input_year].reset_index()
    previous_year_data = input_df[input_df['Date'].dt.year == input_year - 1].reset_index()
    selected_year_data['value_difference'] = selected_year_data.Value.sub(previous_year_data.Value, fill_value=0)
    return pd.concat([selected_year_data.Country, selected_year_data.Value, selected_year_data.value_difference], axis=1).sort_values(by="value_difference", ascending=False)

In [53]:
# Dashboard Main Panel
st.markdown("<h1 style='text-align: center; color: #004d40;'>📦 Import-Export Transactions Dashboard</h1>", unsafe_allow_html=True)
st.markdown("<h4 style='text-align: center; color: #004d40;'>Comprehensive analysis of global trade flows by year and country</h4>", unsafe_allow_html=True)


st.markdown("<hr>", unsafe_allow_html=True)

DeltaGenerator()

In [57]:
# Display key metrics in card layout
df_transaction_diff_sorted = calculate_transaction_difference(df, selected_year)

first_country = df_transaction_diff_sorted.Country.iloc[0] if selected_year > df['Date'].dt.year.min() else '-'
first_country_value = format_value(df_transaction_diff_sorted.Value.iloc[0])
first_country_delta = format_value(df_transaction_diff_sorted.value_difference.iloc[0]) if selected_year > df['Date'].dt.year.min() else '-'

last_country = df_transaction_diff_sorted.Country.iloc[-1] if selected_year > df['Date'].dt.year.min() else '-'
last_country_value = format_value(df_transaction_diff_sorted.Value.iloc[-1])
last_country_delta = format_value(df_transaction_diff_sorted.value_difference.iloc[-1]) if selected_year > df['Date'].dt.year.min() else '-'

In [59]:
# Metrics in a card layout
st.markdown(f"""
<div class='metric-container'>
    <div class='metric-box'>
        <div class='metric-title'>Top Country by Value</div>
        <div class='metric-value'>{first_country}</div>
        <div class='metric-delta'>Difference: {first_country_delta}</div>
    </div>
    <div class='metric-box'>
        <div class='metric-title'>Lowest Country by Value</div>
        <div class='metric-value'>{last_country}</div>
        <div class='metric-delta'>Difference: {last_country_delta}</div>
    </div>
</div>
""", unsafe_allow_html=True)

st.markdown("<hr>", unsafe_allow_html=True)

DeltaGenerator()

In [61]:
# New Insights Section
st.markdown("### Key Insights:")
insight_1 = f"*Top Country*: {first_country} had the highest transaction value of {first_country_value}, reflecting its dominance in the selected trade data."
insight_2 = f"*Bottom Country*: {last_country} had the lowest transaction value of {last_country_value}, indicating it contributed less to the overall trade."
insight_3 = f"The selected countries combined have seen varied growth across trade categories, with noticeable peaks and troughs across specific time frames."
st.write(insight_1)
st.write(insight_2)
st.write(insight_3)

In [63]:
# Additional Graphs Section
col1, col2 = st.columns(2)

with col1:
    st.markdown('#### Top 10 Countries by Transaction Value')
    
    # Bar chart for top countries
    top_10_countries = df_selected_year.groupby('Country')['Value'].sum().reset_index().sort_values(by='Value', ascending=False).head(10)
    bar_chart = alt.Chart(top_10_countries).mark_bar().encode(
        x=alt.X('Value:Q', title='Transaction Value'),
        y=alt.Y('Country:N', sort='-x', title='Country'),
        color=alt.Color('Country:N', scale=alt.Scale(scheme=selected_color_theme))
    ).properties(width=600, height=400)
    st.altair_chart(bar_chart, use_container_width=True)

with col2:
    st.markdown('#### Transactions Over Time')

In [65]:
# Line chart for transactions over time
line_chart = alt.Chart(df_selected_year).mark_line().encode(
x=alt.X('Date:T', axis=alt.Axis(title='Date')),
y=alt.Y('Value:Q', axis=alt.Axis(title='Transaction Value')),
color=alt.Color('Category:N', legend=alt.Legend(title="Category"))
).properties(width=600, height=400)
st.altair_chart(line_chart, use_container_width=True)

st.markdown("<hr>", unsafe_allow_html=True)

DeltaGenerator()

In [67]:
# Pie chart for value distribution by category
st.markdown('#### Value Distribution by Category')

pie_chart = px.pie(df_selected_year, values='Value', names='Category', title="Value Distribution by Category",
                   color_discrete_sequence=px.colors.sequential.RdBu)
pie_chart.update_traces(textinfo='percent+label')
st.plotly_chart(pie_chart, use_container_width=True)

DeltaGenerator()

In [69]:
# Heatmap for transactions by category over time
st.markdown('#### Category-Wise Heatmap')

heatmap = alt.Chart(df_selected_year).mark_rect().encode(
    y=alt.Y('Category:O', axis=alt.Axis(title="Category")),
    x=alt.X('Date:T', axis=alt.Axis(title="Date")),
    color=alt.Color('Value:Q', scale=alt.Scale(scheme=selected_color_theme)),
).properties(width=900, height=300)
st.altair_chart(heatmap, use_container_width=True)

DeltaGenerator()